# Data preprocessing

In [1]:
#%pip install spacy

In [2]:
#%pip install spacy-langdetect

## 1) Data loading

In [3]:
import pandas as pd

In [4]:
# Read CSV file with reviews into a DataFrame
path = '../data/Reviews.csv'
reviews = pd.read_csv(path)

In [5]:
reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [6]:
# Drop customer profile name as this is not useful and even if we need to identify customers we can do it through the unique UserId
reviews.drop("ProfileName", axis=1, inplace=True)

In [7]:
reviews.head()

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [8]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   HelpfulnessNumerator    568454 non-null  int64 
 4   HelpfulnessDenominator  568454 non-null  int64 
 5   Score                   568454 non-null  int64 
 6   Time                    568454 non-null  int64 
 7   Summary                 568427 non-null  object
 8   Text                    568454 non-null  object
dtypes: int64(5), object(4)
memory usage: 39.0+ MB


## 2) Preprocessing

### Type conversion

In [9]:
# Convert ProductId, UserId, Summary and Text to string format
reviews['ProductId'] = reviews['ProductId'].astype('str')
reviews['UserId'] = reviews['UserId'].astype('str')
reviews['Summary'] = reviews['Summary'].astype('str')
reviews['Text'] = reviews['Text'].astype('str')

### Renaming columns

In [10]:
# Renaming columns
reviews = reviews.rename(columns={'Id': 'id', 'ProductId': 'product_id', 'UserId': 'user_id', 'HelpfulnessNumerator': 'helpfulness_numerator',
                                  'HelpfulnessDenominator': 'helpfulness_denominator', 'Score': 'score', 'Time': 'time', 'Summary': 'summary',
                                  'Text': 'text'})

In [11]:
reviews.head()

,id,product_id,user_id,helpfulness_numerator,helpfulness_denominator,score,time,summary,text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


### Adding extra columns

* date from timestamp
* sentiment from score
* helpfulness ratio

In [12]:
# Convert timestamp to date
reviews['date'] = pd.to_datetime(reviews['time'], unit='s')


In [13]:
reviews.head()

,id,product_id,user_id,helpfulness_numerator,helpfulness_denominator,score,time,summary,text,date
0,1,B001E4KFG0,A3SGXH7AUHU8GW,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,2011-04-27
1,2,B00813GRG4,A1D87F6ZCVE5NK,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,2012-09-07
2,3,B000LQOCH0,ABXLMWJIXXAIN,1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,2008-08-18
3,4,B000UA0QIQ,A395BORC6FGVXV,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,2011-06-13
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,2012-10-21


In [14]:
# Check min and max values of date column to check if datetime conversions are ok
print(min(reviews['date']))
print(max(reviews['date']))

1999-10-08 00:00:00
2012-10-26 00:00:00


Dates output seems consistent with the starting date referenced by the author of the dataset in https://snap.stanford.edu/data/web-FineFoods.html

In [15]:
def sentiment_from_score(score):
    if score < 3 and score >= 1:
        return 'negative'
    elif score == 3:
        return 'neutral'
    elif score > 3:
        return 'positive'
    else:
        return 'NA'

reviews['sentiment'] = reviews.apply(lambda row: sentiment_from_score(row['score']), axis=1)

In [16]:
reviews['helpfulness_ratio'] = reviews['helpfulness_numerator'] / reviews['helpfulness_denominator']

### Check for empty reviews

In [17]:
reviews[reviews['text']==""].shape[0]

0

There don't seem to be any records with empty text reviews.

### Analysis for duplicates

In [18]:
reviews['text'].duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
568449    False
568450    False
568451    False
568452    False
568453    False
Name: text, Length: 568454, dtype: bool

In [19]:
reviews['duplicated'] = reviews['text'].duplicated()

In [20]:
# Check if there are any duplicates in the review text
print(f"# of duplicated records in the review text: {sum(reviews['text'].duplicated())}")

# Get reviews that have the same text and product_id
print(f"# of reviews with the same product_id and text: {sum(reviews.duplicated(subset=['product_id','text']))}")


# Get reviews that have the same text, user_id and product_id
print(f"# of reviews with the same user_id, product_id and text: {sum(reviews.duplicated(subset=['user_id','product_id','text']))}")


# of duplicated records in the review text: 174875
# of reviews with the same product_id and text: 1324
# of reviews with the same user_id, product_id and text: 1309


Not only are there duplicated reviews for the same product but from different users, but also duplicated reviews that concern the same user and product. We should at least remove the ones that have the same user_id, product_id and text. One way to do it would be to keep either the first or the latest review among these duplicates.

It may make more sense to just remove reviews that have the same user_id, product_id and text since it could happen that some duplicate reviews are short sentences that more than one person can write or a user may agree with another and copy their review. In the latter case, this review accurately reflects the user's opinion and he simply didn't bother to write an original text review.

In [21]:
# Drop record duplicates with the same user_id, product_id and review text keeping only the earliest or latest review

def drop_same_user_product_text(df, keep='first'):
    
    keep_mask = df.sort_values(by='date', ascending=True).duplicated(subset=['user_id','product_id','text'], keep=keep)

    df = df[keep_mask]

    return df

In [22]:
# Drop record duplicates with the same review text for the same product_id keeping only the earliest or latest review

def drop_same_product_text(df, keep='first'):

    keep_mask = df.sort_values(by='date', ascending=True).duplicated(subset=['product_id','text'], keep=keep)

    df = df[keep_mask]

    return df

In [23]:
reviews = drop_same_user_product_text(reviews, keep='first')

C:\Users\j.martins\AppData\Local\Temp\ipykernel_15740\4254619130.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[keep_mask]


In [24]:
reviews.head()

,id,product_id,user_id,helpfulness_numerator,helpfulness_denominator,score,time,summary,text,date,sentiment,helpfulness_ratio,duplicated
466,467,B000G6RYNE,A3PJZ8TU8FDQ1K,0,0,5,1231718400,"One bite and you'll become a ""chippoisseur""","I'm addicted to salty and tangy flavors, so wh...",2009-01-12,positive,NaN,False
574,575,B000G6RYNE,A3PJZ8TU8FDQ1K,2,2,5,1231718400,"One bite and you'll become a ""chippoisseur""","I'm addicted to salty and tangy flavors, so wh...",2009-01-12,positive,1.0,True
2334,2335,B0001FQVCK,A5D06XJHDXK75,3,3,5,1190592000,Chocolate Italian kisses - need I say more?,My family has been in love with Baci's ever si...,2007-09-24,positive,1.0,False
2613,2614,B0016FY6H6,A3I4PCBRENJNG2,4,4,4,1300579200,lots of uses for this green tea...,lots of uses for this green tea...<br /><br />...,2011-03-20,positive,1.0,False
2647,2648,B0016FY6H6,A2NLZ3M0OJV9NX,0,0,2,1313107200,way too weak for my tastes,"What can i say, I love iced tea. I drink a lot...",2011-08-12,negative,NaN,True


### Double check if all reviews are in english

In [46]:
# Run this to download language model

#!python -m spacy download en

In [35]:
#from spacy_langdetect import LanguageDetector
#import spacy
#from spacy.language import Language

In [ ]:
#nlp = spacy.load("en_core_web_sm")

#@Language.factory("language_detector")
#def get_lang_detector(nlp, name):
#    return LanguageDetector()
#
#nlp.add_pipe('language_detector', last=True)

In [51]:
#def is_english(text):
#
#    language_score = nlp(text)._.language
#
#    if language_score['language'] == 'en' and language_score['score'] > 0.5:
#        return True
#    else:
#        return False

In [52]:
#reviews['english'] = reviews.apply(lambda row: is_english(row['text']), axis=1)

In [59]:
# Count number of non-english reviews
#print(f'# of non-english reviews = {reviews.shape[0]-sum(reviews["english"])}')

# of non-english reviews = 0


The above code confirms that there are no non-english reviews but is slow to run so after running one time we can comment it.

## 3) Exploratory data analysis